In [1]:
import csv
import copy

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from consul.agent_types.rl_agent import RLAgent
from consul.agent_types.cbr_agent import CaseBased_RLAgent, get_float_dict
from war_reason import *
from war_reason_implementations import *

# Load the data

In [2]:
data = pd.read_pickle("FullData.pickle")

In [3]:
# The Game-Theoretic Equilibrium
equilibrium_cols = ['eqTsq', 'eqTnego', 'eqTacqa', 'eqTacqb', 'eqTcapa', 'eqTcapb', 
                'eqTwara', 'eqTwarb']

data["Equilibrium"] = ""
for col in equilibrium_cols:
    data.loc[data[col] == 1, "Equilibrium"] = col

# Cutting out missing data
data = data[data.Equilibrium!=""]

In [4]:
# Observed outcomes
data["Outcome"] = ""

data.loc[((data.cwhost1<2) & (data.cwhost2<2)), "Outcome"] = "StatusQuo"
data.loc[((data.cwhost1>3) & (data.cwhost2>3)), "Outcome"] = "War"
data.loc[((data.cwhost1==data.cwhost2) & (data.cwhost1>1) & 
          (data.cwhost1<4)), "Outcome"] = "Negotiation"
data.loc[((data.cwhost1>data.cwhost2) & (data.cwhost1>1) & 
          (data.cwhost1<4)), "Outcome"] = "Acquiesce_B"
data.loc[((data.cwhost1<data.cwhost2) & (data.cwhost2>1) & 
          (data.cwhost2<4)), "Outcome"] = "Acquiesce_A"
data.loc[((data.cwhost1>data.cwhost2) & (data.cwhost1>3)), "Outcome"] = "Capitulate_B"
data.loc[((data.cwhost1<data.cwhost2) & (data.cwhost2>3)), "Outcome"] = "Capitulate_A"

In [5]:
# OPTIONAL
## Boost all utilities by a fixed factor
for col in data.columns:
    if col[:4] == "wrTu":
        data[col] *= 8

In [6]:
def get_outcome(m):
    return m.current_node

# Set up parallel

In [10]:
import ipyparallel

In [12]:
clients = ipyparallel.Client()
print(clients.ids)
dview = clients[:]

[0, 1, 2]


In [14]:
with dview.sync_imports():
    from war_reason_implementations import assess_moves
    from consul.agent_types.cbr_agent import CaseBased_RLAgent, get_float_dict
    from rtree import index

importing assess_moves from war_reason_implementations on engine(s)
importing CaseBased_RLAgent,get_float_dict from consul.agent_types.cbr_agent on engine(s)
importing index from rtree on engine(s)


In [15]:
dview.push({
            #"data": data,
           "EUGeneWorld": EUGeneWorld,
           "RLAgent": RLAgent,
           #"CaseBased_RLAgent": CaseBased_RLAgent,
           #"get_float_dict": get_float_dict,
           "CBRAgent": CBRAgent,
           "get_outcome": get_outcome})

<AsyncResult: finished>

In [16]:
eugene_world = EUGeneWorld(CBRAgent, data, 
                       agent_args={"learning_rate": 0.2, "discount_factor": 0.8},
                       copy_data=True, shuffle_rows=True)
inputs = [copy.deepcopy(eugene_world) for _ in range(4)]

In [17]:
def run_model(model):
    #return None
    model.run(report_every=None,
              model_assessors={"Model_Outcome": get_outcome,
                               "Move_Quality": assess_moves})
    #return model
    return model.eugene_data

In [ ]:
runs = dview.map_sync(run_model, inputs)

In [ ]:
for i, df in enumerate(runs):
    df["Iteration"] = i

In [ ]:
concat_df = pd.concat(runs)

In [ ]:
concat_df.head()

In [ ]:
concat_df.columns

In [ ]:
cols_to_keep = ['year', "ccode1", "ccode2", "Iteration",
                 "Outcome", "Equilibrium", "Model_Outcome", "Move_Quality"]
concat_df = concat_df[cols_to_keep]

In [ ]:
concat_df.to_pickle("Model_Outputs.pickle")

In [ ]:
200 / 60